In [ ]:
import mediapipe as mp
import os
import json

model_path = '../packages/efficientdet_lite0.tflite'
keyframes_base_path = '../keyframe_information/keyframe'
output_base_path = '../keyframe_information/object_metadata'
BaseOptions = mp.tasks.BaseOptions
ObjectDetector = mp.tasks.vision.ObjectDetector
ObjectDetectorOptions = mp.tasks.vision.ObjectDetectorOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = ObjectDetectorOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    max_results=5,
    running_mode=VisionRunningMode.IMAGE
)

for video_folder in os.listdir(keyframes_base_path):
    video_folder_path = os.path.join(keyframes_base_path, video_folder)
    if os.path.isdir(video_folder_path) and '_reduced' not in video_folder:
        for keyframe_subfolder in os.listdir(video_folder_path):
            keyframe_subfolder_path = os.path.join(video_folder_path, keyframe_subfolder)
            
            if os.path.isdir(keyframe_subfolder_path):
                output_subfolder_path = os.path.join(output_base_path, video_folder, keyframe_subfolder)
                os.makedirs(output_subfolder_path, exist_ok=True)

                with ObjectDetector.create_from_options(options) as detector:
                    for filename in os.listdir(keyframe_subfolder_path):
                        if filename.endswith('.jpg'):
                            keyframe_path = os.path.join(keyframe_subfolder_path, filename)
                            try:
                                mp_image = mp.Image.create_from_file(keyframe_path)
                            except Exception as e:
                                print(f"Error reading image {keyframe_path}: {e}")
                                continue
                            detection_result = detector.detect(mp_image)

                            if not detection_result.detections:
                                print(f"No detections for {filename}")
                                continue 

                            detection_list = []
                            for detection in detection_result.detections:
                                detection_data = {
                                    'bounding_box': {
                                        'xmin': detection.bounding_box.origin_x,
                                        'ymin': detection.bounding_box.origin_y,
                                        'width': detection.bounding_box.width,
                                        'height': detection.bounding_box.height
                                    },
                                    'score': detection.categories[0].score,
                                    'category_name': detection.categories[0].category_name
                                }
                                detection_list.append(detection_data)

                            output_path = os.path.join(output_subfolder_path, f'{os.path.splitext(filename)[0]}.json')
                            try:
                                with open(output_path, 'w') as output_file:
                                    json_data = json.dumps(detection_list, indent=4)
                                    output_file.write(json_data)
                                print(f"Saved detection result for {filename} to {output_path}")
                            except Exception as e:
                                print(f"Error writing JSON for {filename}: {e}")
